In [ ]:
!pip install azure-mgmt-datafactory

     |████████████████████████████████| 517 kB 5.5 MB/s 
     |████████████████████████████████| 41 kB 492 kB/s 
     |████████████████████████████████| 193 kB 7.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-25c7b621-6100-4c63-bff1-7a73055ca94c
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-25c7b621-6100-4c63-bff1-7a73055ca94c/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install azure-identity

     |████████████████████████████████| 166 kB 5.1 MB/s 
     |████████████████████████████████| 102 kB 1.3 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-25c7b621-6100-4c63-bff1-7a73055ca94c/bin/python -m pip install --upgrade pip' command.


In [ ]:
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
import pandas as pd
import json

In [ ]:
# Configurar credenciais
id_cliente = '' # ID do aplicativo (cliente)
segredo_cliente = '' #Segredo do Cliente
id_diretorio = '' #ID do diretório (locatário)

# Substitua os valores abaixo com suas informações
id_assinatura = ''
grupo_recurso = ''
nome_data_factory = ''

In [ ]:
def conectar_adf(id_cliente, segredo_cliente, id_diretorio, id_assinatura):
    
    credentials = ClientSecretCredential(client_id=id_cliente, client_secret=segredo_cliente, tenant_id=id_diretorio) 

    adf_client = DataFactoryManagementClient(credential=credentials, subscription_id=id_assinatura)
    return adf_client

In [ ]:
def extrair_pipelines(adf_client, grupo_recurso, nome_data_factory):

    pipelines = adf_client.pipelines.list_by_factory(resource_group_name=grupo_recurso, factory_name=nome_data_factory)

    pipe_id = 0
    dados_piplines = []
    dados_inputs = []
    dados_outputs = []
    for pipeline in pipelines:
        pipe_id += 1
        nome_pipeline = pipeline.name
        descricao = pipeline.description

        dados_dict = {
            "ID": pipe_id,
            "NOME": nome_pipeline,
            "DESCRICAO": descricao}
        dados_piplines.append(dados_dict)

        pipeline_dict = pipeline.serialize()

        response = adf_client.pipelines.get(grupo_recurso, nome_data_factory, pipeline_name=f"{nome_pipeline}")

        activities = response.serialize()
        for activity in activities["properties"]["activities"]:
            act_name = activity['name']
            act_inputs = activity.get('inputs', [])
            act_outputs = activity.get('outputs', [])
            query = activity.get('typeProperties', {}).get('source', {}).get('sqlReaderQuery', '')
            sink_script = activity.get('typeProperties', {}).get('sink', {}).get('preCopyScript', '')

            active_info_origem = {
                "ID": pipe_id,
                "NOME": act_name,
                "TIPO" : act_inputs[0]["type"] if act_inputs else None,
                "CONJUNTO_DADOS" : act_inputs[0]["referenceName"] if act_inputs else None,
                "PARAMETROS" : act_inputs[0]["parameters"] if act_inputs else None,
                "QUERY": query
            }
            active_info_coletor = {
                "ID": pipe_id,
                "NOME": act_name,
                "TIPO" : act_outputs[0]["type"] if act_outputs else None,
                "CONJUNTO_DADOS" : act_outputs[0]["referenceName"] if act_outputs else None,
                "PARAMETROS" : act_outputs[0]["parameters"] if act_outputs else None,
                "SCRIPT": sink_script
            }
            dados_inputs.append(active_info_origem)
            dados_outputs.append(active_info_coletor)

    return dados_piplines, dados_inputs, dados_outputs

In [ ]:
#chamando funções
adf_client = conectar_adf(id_cliente, segredo_cliente, id_diretorio, id_assinatura)
dados_piplines, dados_inputs, dados_outputs = extrair_pipelines(adf_client, grupo_recurso, nome_data_factory)

In [ ]:
#Transformando dados em json
dados_inputs_json = json.dumps(dados_inputs)
dados_outputs_json = json.dumps(dados_outputs)
dados_piplines_json = json.dumps(dados_piplines)

In [ ]:
df = pd.read_json(dados_piplines_json)
display(df)

ID,NOME,DESCRICAO
1,PL_copy_csv_db_cadastros,Cópia do csv -datalake para o banco sql
2,PL_copy_csv_produtos_vendas,null
3,copia_tabela_schema_dbo,Copia tabela de vendas do schema empresa para o schema dbo


In [ ]:
df_inp = pd.read_json(dados_inputs_json)
df_inp['PARAMETROS'] = df_inp['PARAMETROS'].astype("string")

df_inp['PARAMETROS'] = df_inp['PARAMETROS'].str.replace('{', '[', regex=False)
df_inp['PARAMETROS'] = df_inp['PARAMETROS'].str.replace('}', ']', regex=False)
df_inp['PARAMETROS'] = df_inp['PARAMETROS'].str.replace(',', '] [', regex=False)
df_inp['PARAMETROS'] = df_inp['PARAMETROS'].str.replace("'", "", regex=False)

display(df_inp) 

ID,NOME,TIPO,CONJUNTO_DADOS,PARAMETROS,QUERY
1,copia csv cliente,DatasetReference,DL_dinamico_virgula,[file_name: CadastroClientes.csv],
1,copia csv - Lojas,DatasetReference,DL_dinamico_virgula,[file_name: Contoso - Lojas.csv],
1,copia csv -funcionarios,DatasetReference,DL_dinamico_ponto_virgula,[file_name: CadastroFuncionarios.csv],
2,copia csv produtos,DatasetReference,DL_dinamico_ponto_virgula,[file_name: Contoso - Cadastro Produtos.csv],
2,inner vendas,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: vendas],"SELECT v.[Numero da Venda], v.[Data da Venda], v.[Quantidade Vendida], v.[Quantidade Devolvida], p.[Nome da Marca], p.Tipo, p.Categoria, p.[Preco Unitario], l.[Nome da Loja] FROM empresa.vendas v LEFT JOIN empresa.lojas l ON l.[ID Loja] =v.[ID Loja] LEFT JOIN empresa.produtos p ON p.[ID Produto] = v.[ID Produto]"
3,copy_tabela_vendas_schema_dbo,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: vendas],"SELECT v.[Numero da Venda], v.[Data da Venda], v.[Quantidade Vendida], v.[Quantidade Devolvida], p.[Nome da Marca], p.Tipo, p.Categoria, p.[Preco Unitario], l.[Nome da Loja] FROM empresa.vendas v LEFT JOIN empresa.lojas l ON l.[ID Loja] =v.[ID Loja] LEFT JOIN empresa.produtos p ON p.[ID Produto] = v.[ID Produto]"


In [ ]:
df_out = pd.read_json(dados_outputs_json)
df_out['PARAMETROS'] = df_out['PARAMETROS'].astype("string")


df_out['PARAMETROS'] = df_out['PARAMETROS'].str.replace('{', '[', regex=False)
df_out['PARAMETROS'] = df_out['PARAMETROS'].str.replace('}', ']', regex=False)
df_out['PARAMETROS'] = df_out['PARAMETROS'].str.replace(',', '] [', regex=False)
df_out['PARAMETROS'] = df_out['PARAMETROS'].str.replace("'", "", regex=False)

display(df_out)

ID,NOME,TIPO,CONJUNTO_DADOS,PARAMETROS,SCRIPT
1,copia csv cliente,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: clientes],truncate table empresa.clientes
1,copia csv - Lojas,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: lojas],
1,copia csv -funcionarios,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: funcionarios],
2,copia csv produtos,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: produtos],truncate table empresa.produtos
2,inner vendas,DatasetReference,sql_dinamico,[schema_name: empresa] [ table_name: juncao_vendas],
3,copy_tabela_vendas_schema_dbo,DatasetReference,AzureSqlTable_dbo_clientes,[],
